In [40]:
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [112]:
def MyFConv2D(input, weight, bias=None, stride=1, padding=0):
    
    """
    My custom Convolution 2D calculation.

    [input]
    * input    : (batch_size, in_channels, input_height, input_width)
    * weight   : (you have to derive the shape :-)
    * bias     : bias term
    * stride   : stride size
    * padding  : padding size

    [output]
    * output   : (batch_size, out_channels, output_height, output_width)
    """

    assert len(input.shape) == len(weight.shape) , "weight shape incorrect"
    assert len(input.shape) == 4, "input shape incorrect"
        
    # batch_size, in_channels, input_height, input_width = input.shape
    N, C_in, H_in, W_in = input.shape
    # weight shape  (C_out, C_in, kH, kW)
    C_out, C_in, kH, kW = weight.shape 
    bias, s, p = bias, stride, padding

    x_pad =  F.pad(input,(p,p,p,p),mode='constant',value=0) # add 2p to last two dims of input
    assert x_pad.shape == (N, C_in, H_in + 2*p, W_in + 2*p) , "padding incorrect"

    H_out = int( (H_in + 2*p - kH) / s) + 1 # output sizes 
    W_out = int( (W_in + 2*p - kW) / s) + 1
    
    output = torch.zeros(N, C_out, H_out, W_out) # initalise output tensor
    
    ## Convolution
    for h in range(H_out):
        for w in range(W_out):
            
            window = x_pad[:, :, h*s:h*s+kH, w*s:w*s+kW]
            # (b,c_in, kH, KW) * (c_o, c_i, kH, kW) -> (b, c_o, _)
            # sum over c_in, kH, kW
            output[:,:, h, w] = torch.tensordot(window, weight, dims=([1,2,3],[1,2,3]))
            if bias is not None:
                output[:,:, h, w] += bias[:]
    return output


In [113]:


class MyConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, bias=True):

        """
        My custom Convolution 2D layer.

        [input]
        * in_channels  : input channel number
        * out_channels : output channel number
        * kernel_size  : kernel size
        * stride       : stride size
        * padding      : padding size
        * bias         : taking into account the bias term or not (bool)

        """
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias

        ## Create the torch.nn.Parameter for the weights and bias (if bias=True)
        ## Be careful about the size
        # weight shape  (C_out, C_in, kH, kW)
        # N, C_in, H_in, W_in = input.shape

        self.W =  nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        self.b = nn.Parameter(torch.zeros(out_channels)) if bias else None

        # out = ( (2*p + H - k) / s ) + 1            
    
    def __call__(self, x):
        
        return self.forward(x)


    def forward(self, x):
        
        """
        [input]
        x (torch.tensor)      : (batch_size, in_channels, input_height, input_width)

        [output]
        output (torch.tensor) : (batch_size, out_channels, output_height, output_width)
        """

        # call MyFConv2D here
        output = MyFConv2D(x, self.W, self.b, self.stride, self.padding)
        return output

In [118]:

# Create inputs for validation
batch_size, in_channels, H, W = 4,5,8,10 # Example dimensions
out_channels = 3
kernel_size = 3
stride = 1
padding = 2

# Input and weights
input_tensor = torch.rand(batch_size, in_channels, H, W)
weights = torch.rand(out_channels, in_channels, kernel_size, kernel_size)
bias = torch.zeros(out_channels)

# custom_conv_output = MyFConv2D(input_tensor, weights, bias, stride, padding)

myConvd2D = MyConv2D(in_channels, out_channels, kernel_size, stride, padding)


conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
conv2d.weight.data = myConvd2D.W.data.clone()
conv2d.bias.data = myConvd2D.b.data.clone()

# Apply PyTorch Conv2D
with torch.no_grad():  # Ensure Conv2D weight and bias are not updated
    torch_conv_output = conv2d(input_tensor)


my_conv_output = myConvd2D(input_tensor)

# Compare outputs
print("Output Difference:", torch.sum(torch_conv_output - my_conv_output))


Output Difference: tensor(0., grad_fn=<SumBackward0>)


In [125]:
input_tensor.shape

torch.Size([4, 5, 8, 10])

In [132]:
a = b = 0
a = 1

In [133]:
b

0